<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/main/KagglePrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAVORO PER SARDE:
1. Aggiungere altri todo
2. Imparare ad essere umile
3. Tifare il Campobasso

TODO: CONTROLLARE I PARAGRAFI

1. Controllare se non ci sono whitespace dopo gli .\n Maiuscola
2. Score value spacy per identificare la similarità, potremmo usarla per paragrafi diversi.

TODO: CONTROLLARE I NOMI PROPRI
1. Si potrebbe effettuare il lower() prima di aggiungere le parole allo spellchecker sperando che ci siano i nomi propri necessari alla stesura del riassunto all'interno del prompt
2. Controllare i nomi propri facendo ner e non tenerli in considerazione per eventuali errori, ma confrontarli con eventuali nomi propri all'interno dei prompt (selezionare i soggetti che iniziano per le stesse lettere e poi calcolarcisi l'edit distance ==> wording, entro un certo range viene considerato match e si salva una feature per il numero di match con i prompt ==> content)



## Data importing and setup

In [37]:
!pip install pyspellchecker symspellpy swifter

In [38]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

fatal: destination path 'CommonLitChallenge' already exists and is not an empty directory.


In [39]:
import pandas as pd
import swifter

summaries_train_path = "/content/CommonLitChallenge/data/summaries_train.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

## Language Processing

> Definition of the Glossary \\
> Creation of the word list \\
> Language Model Class construction

### Creating the list of words

In [40]:
import itertools
word_list = []
for prompt, query, title in zip(prompt_data.prompt_text.tolist(), prompt_data.prompt_question.tolist(), prompt_data.prompt_title.tolist()):
  word_list.append(prompt.replace('\n', ' ').split())
  word_list.append(query.replace('\n', ' ').split())
  word_list.append(title.replace('\n', ' ').split())
token_list = list(itertools.chain(*word_list))

### Language Model from Spacy


In [41]:
import numpy as np
import pkg_resources
from symspellpy import SymSpell

class LanguageMagician():
  def __init__(self) -> None:
    self.gamma = 3e-3
    self.modulus = 541

    # Initialization of SymSpell
    self.sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt"
    )
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
    )
    # term_index is the column of the term and count_index is the
    # column of the term frequency
    self.sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    self.sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

  def init_vocabulary_(self, words):
    vector_data = {word: np.random.uniform(-1, 1, (300,)) for word in words}
    vocab = self.nlp.vocab
    for word, vector in vector_data.items():
        vocab.set_vector(word, vector)

  def init_sym_vocabulary(self, words):
    #open text file
    text_file = open("vocabulary.txt", "a")
    #write string to file
    for s in words:
      text_file.write(s)

    #close file
    text_file.close()

    corpus_path = 'vocabulary.txt'
    try:
      result = self.sym_spell.create_dictionary(corpus_path)
    except:
      raise(f'error the dictionary creation has returned {result}')


  def correct(self, row):
    # max edit distance per lookup (per single word, not per whole input string)
    suggestions = self.sym_spell.lookup_compound(row, max_edit_distance=3,
                                                #  transfer_casing=True
                                                 )
    # display suggestion term, edit distance, and term frequency
    return suggestions[0].term, suggestions[0].distance


lang_wizard = LanguageMagician()
lang_wizard.init_sym_vocabulary(token_list)
lang_wizard.init_sym_vocabulary(token_list)

In [42]:
train_data[["corrected_text", "distance"]] = train_data.swifter.apply(lambda x: lang_wizard.correct(x.text), axis=1, result_type='expand')

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

## Text processing

In [43]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessText(text, removal=True):
    # replace newline with space
    text = text.replace("\n", " ")
    text = text.replace('\r', ' ')

    # Normalize spaces around punctuation marks
    text = re.sub(r"[^A-Za-z0-9']", r' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    text = text.strip()

    if removal:
      # lower case
      text = text.lower()

      # split text
      words = text.split()

      # stop word removal
      words = [w for w in words if not w in stop_words]

      # stemming
      # words = [stemmer.stem(w) for w in words]

      # lemmatization
      words = [lemmatizer.lemmatize(w) for w in words]

      text = ' '.join(words)

    return text

# Preprocess the text
prompt_data["prompt_text_preprocessed"] =   prompt_data["prompt_text"].swifter.apply(lambda x: preprocessText(x))
train_data["corrected_text_preprocessed"] = train_data["corrected_text"].swifter.apply(lambda x: preprocessText(x))

train_data["text_pre_withstop"] =           train_data["text"].swifter.apply(lambda x: preprocessText(x, False))
train_data["corrected_text_pre_withstop"] = train_data["corrected_text"].swifter.apply(lambda x: preprocessText(x, False))
# Count the length of text
prompt_data["prompt_text_length"] = prompt_data["prompt_text"].swifter.apply(len)
train_data["text_length"] =         train_data["corrected_text"].swifter.apply(len)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### N-grams finding

In [44]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
del_columns = []

for n in range(2,5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.swifter.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-correctedtext-count"
  train_data[col] = train_data.swifter.apply(lambda row: count_ngrams(row["corrected_text"], n), axis=1)
  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.swifter.apply(lambda row: count_ngrams(row["text"], n), axis=1)
  del_columns.append(col)

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Different Words counter

In [46]:
def different_word_counter(row):
  row = preprocessText(row)
  words_list = []
  for w in row.split(' '):
    if w not in words_list:
      words_list.append(w)

  return len(words_list)


In [47]:
prompt_data["prompt_text_word_counter"] = prompt_data['prompt_text'].swifter.apply(lambda row: different_word_counter(row))
train_data['summary_word_counter'] = train_data['corrected_text'].swifter.apply(lambda row: different_word_counter(row))

Pandas Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

In [48]:
# @title Merging the dataset
training_data = train_data.merge(prompt_data, on='prompt_id')

### Different words counter ratio

In [49]:
def word_counter_ratio(summary_word_counter, prompt_word_counter):
  return summary_word_counter/prompt_word_counter

In [50]:
training_data['word_counter_ratio'] = training_data.swifter.apply(lambda row: word_counter_ratio(row["summary_word_counter"], row["prompt_text_word_counter"]), axis=1)

### N-grams counting

In [51]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [52]:
for n in range(2, 5):

  text_col = f"{n}grams-text-count"
  corre_col = f"{n}grams-correctedtext-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-correct-count"
  training_data[new_col] = training_data.swifter.apply(lambda row: count_cooccurring_ngrams(row[text_col], row[corre_col]), axis=1)

  new_col=f"{n}grams-cooccurrence-count"
  training_data[new_col] = training_data.swifter.apply(lambda row: count_cooccurring_ngrams(row[corre_col], row[prompt_col]), axis=1)

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7165 [00:00<?, ?it/s]

### Text length ratio

In [53]:
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

### TF-IDF score creation

In [54]:
def add_row(df1, df2, preprocess=False):
  row = df2.unique().tolist()[0]
  if preprocess:
    row = preprocessText(row)
  combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe
  return combined_data

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Group by 'prompt_id' and compute TF-IDF separately for each class
tfidf_vectorizers = {}

for class_id, group in training_data.groupby('prompt_id'):
    #text_data = group['text'].swifter.apply(preprocessText)
    text_data = group['corrected_text_preprocessed']

    prompt_question_data = group['prompt_question']
    prompt_title_data = group['prompt_title']
    prompt_text_data = group['prompt_text_preprocessed']

    # Concatenate the preprocessed data for TF-IDF calculation
    combined_data = add_row(text_data, prompt_question_data, True)
    combined_data = add_row(combined_data, prompt_title_data, True)
    combined_data = add_row(combined_data, prompt_text_data)


    # Compute TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

In [56]:
karp_tfidf_scores = {}

# Calculate TF-IDF scores for each document
for class_id, group in training_data.groupby('prompt_id'):
    tfidf_vectorizer = tfidf_vectorizers[class_id]['vectorizer']
    tfidf_matrix = tfidf_vectorizers[class_id]['matrix']
    tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title

    modulus = 541

    # Iterate through documents and calculate TF-IDF scores
    for index, row in group.iterrows():
        doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

        doc_tfidf = doc_tfidf[doc_tfidf>0]

        # Calculate the average TF-IDF score for the document
        #average_tfidf_score = sum(doc_tfidf) / len(doc_tfidf)

        #average_tfidf_scores[index] = average_tfidf_score

        gamma = 1e-2
        single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

        karp_tfidf_scores[index] = single_tfidf_score

In [57]:
# Add the calculated average TF-IDF scores as a new column to the DataFrame
training_data['karp_tfidf_scores'] = [karp_tfidf_scores[index] for index in training_data.index]

## Dataset normalization

In [58]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(df, col):
    # Create a Min-Max Scaler
    scaler = MinMaxScaler()

    df['norma_'+col] = df.groupby('prompt_id')[col].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

    return df

In [59]:
normalize_cols = ["text_length", "distance",
                  "2grams-cooccurrence-count", "2grams-correct-count",
                  "3grams-cooccurrence-count", "3grams-correct-count",
                  "4grams-cooccurrence-count", "4grams-correct-count",
                  "karp_tfidf_scores"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [60]:
normalize_cols.extend(['prompt_text_length', 'prompt_text_word_counter'])
del_columns.extend(normalize_cols)

training_data.drop(columns=del_columns, axis=1, inplace=True)

In [61]:
training_data.head()

,prompt_id,text,content,wording,corrected_text,corrected_text_preprocessed,text_pre_withstop,corrected_text_pre_withstop,summary_word_counter,prompt_question,...,length_ratio,norma_text_length,norma_distance,norma_2grams-cooccurrence-count,norma_2grams-correct-count,norma_3grams-cooccurrence-count,norma_3grams-correct-count,norma_4grams-cooccurrence-count,norma_4grams-correct-count,norma_karp_tfidf_scores
0,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,the third wave was an experiment to see how pe...,third wave experiment see people reacted new o...,The third wave was an experimentto see how peo...,the third wave was an experiment to see how pe...,30,Summarize how the Third Wave developed over su...,...,0.096910,0.107143,0.082569,0.055901,0.107639,0.006757,0.108209,0.000000,0.108,0.502595
1,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,the third wave developed rapidly because the s...,third wave developed rapidly student genuinely...,The Third Wave developed rapidly because the s...,the third wave developed rapidly because the s...,80,Summarize how the Third Wave developed over su...,...,0.335955,0.507049,0.752294,0.198758,0.378472,0.033784,0.328358,0.007463,0.272,0.940274
2,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,the third wave only started as an experiment w...,third wave started experiment within class slo...,The third wave only started as an experiment w...,the third wave only started as an experiment w...,27,Summarize how the Third Wave developed over su...,...,0.095506,0.104793,0.137615,0.093168,0.107639,0.040541,0.111940,0.029851,0.116,0.466262
3,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,the experiment was originally about how even w...,experiment originally even terrible thing happ...,The experimen was orginally about how even whe...,the experiment was originally about how even w...,30,Summarize how the Third Wave developed over su...,...,0.124719,0.153665,0.174312,0.142857,0.142361,0.074324,0.138060,0.052239,0.136,0.516223
4,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,the third wave developed so quickly due to the...,third wave developed quickly due student parti...,The third wave developed so quickly due to the...,the third wave developed so quickly due to the...,13,Summarize how the Third Wave developed over su...,...,0.040449,0.012688,0.055046,0.037267,0.006944,0.006757,0.018657,0.000000,0.032,0.137766


In [62]:
training_data.corrwith(training_data["content"])

content                            1.000000
wording                            0.751380
summary_word_counter               0.791634
word_counter_ratio                 0.802038
length_ratio                       0.777919
norma_text_length                  0.780823
norma_distance                     0.627700
norma_2grams-cooccurrence-count    0.535491
norma_2grams-correct-count         0.765712
norma_3grams-cooccurrence-count    0.388423
norma_3grams-correct-count         0.749101
norma_4grams-cooccurrence-count    0.338160
norma_4grams-correct-count         0.711367
norma_karp_tfidf_scores            0.837687
dtype: float64

In [63]:
training_data.corrwith(training_data["wording"])

content                            0.751380
wording                            1.000000
summary_word_counter               0.516272
word_counter_ratio                 0.560617
length_ratio                       0.548927
norma_text_length                  0.556560
norma_distance                     0.483862
norma_2grams-cooccurrence-count    0.188779
norma_2grams-correct-count         0.548939
norma_3grams-cooccurrence-count    0.035004
norma_3grams-correct-count         0.538261
norma_4grams-cooccurrence-count   -0.001188
norma_4grams-correct-count         0.517875
norma_karp_tfidf_scores            0.566616
dtype: float64

In [64]:
training_data.to_csv('dataset.csv', index=False)

In [65]:
!zip -r 'dataset.zip' 'dataset.csv' # download and save in github

updating: dataset.csv (deflated 95%)
